# Laden und untersuchen von JSON Daten

Viele Inhalte zirkulieren im Web als JSON-Daten, aus denen am Frontend der Endnutzer*innen Inhalte von Webseiten, Plattformen, Apps etc. generiert werden. Im Backend der Zirkulation und Verarbeitung maschinenlesbarer Daten ist JSON ebenfalls sehr wichtig.

In der Python Aufgabe zur Konstruktion einer Sammlung fiktiver Instagram-Posts, haben sie eine ```List``` von ```Dictionaries``` erstellt. Die meisten JSON-Daten sind genau das: Listen (ersichtlich an den ```[ ]``` eckigen Klammern) oder Sammlungen von ähnlich strukturierten Daten über Entitäten. Die Elemente der Liste, die als Dictionaries angelegt worden sind – ersichtlich an den ```{ }``` geschweiften Klammern – haben Attribute, denen Werte zugewiesen sind, die in unserem Fall einen Post repräsentieren.
 
Ausgehend hiervon haben wir uns in der Sitzung eine JSON-Datei angesehen, die Instagram Daten enthält, die mit der Zeeschuimer Firefox Extension erhoben worden sind. Zeeschuimer speichert JSON als NDJSON. Hierbei handelt es sich um eine Variation des JSON Formats, welches bei dem sich je Zeile genau ein vollständiger Eintrag befindet. Die Einträge sind in der Datei also nicht von einer eckigen Klammer umgeben und durch Kommata getrennt. Zeilenumbrüche – die wir uns auch als unsichtbare Steuerzeichen vorstellen können – markieren den Wechsel zwischen Einträgen.

Die Datei befindet sich im Ordner ```fname = '../00_shared_data/Zeeshuimer/zeeschuimer-#gegengewalt-2024-05-07.ndjson```
 

## Bevor es losgeht

Am Anfang laden wir die Bibliotheken, die wir für benötigen. Diese werden per Konvention ganz oben in die erste Code-Zelle geschrieben. Sie können später weitere Einträge hinzufügen und diese durch erneutes Ausführen der Zelle nachladen.

In [ ]:
import json
import pandas as pd 

## Let's start: Varianten des Ladens von JSON bzw. NDJSON Daten

In [ ]:

# Definieren des Dateipfads & -namens als Variable fname
fname = '../00_shared_data/Zeeshuimer/zeeschuimer-#gegengewalt-2024-05-07.ndjson'

# Öffnen der Datei und speichern des Inhalts in der Variable data

data = [] # Anlegen einer leeren Liste
with open(fname) as f: # Öffnen der Datei
    for line in f: # Iterieren über die Zeilen der Datei von denen wir wissen, dass jede einen JSON-String enthält
        entity = json.loads(line) # Laden des JSON-Strings in ein Python-Dictionary
        data.append(entity) # Anhängen des Dictionaries an die Liste data  


# Alternativ können wir das auch in einer Zeile schreiben:
data = [json.loads(line) for line in open(fname)]

len(data) # Ausgabe der Anzahl der Elemente in der Liste data


Das Vorgehen beim öffnen normaler JSON-Dateien ist leicht anders. Hier müssen wir nicht jede Zeile einzeln einlesen, sondern können die Datei als ganze mit der Funktion ```json.load()``` laden.

Nehmen wir eine andere "normale" JSON-Datei als Beispiel: ```../00_shared_data/OpenAI-GPTs/2024-04-05-gpts-en.json```

In [ ]:
with open('../00_shared_data/OpenAI-GPTs/2024-04-05-gpts-en.json', 'r') as f:
    data2 = json.load(f)

len(data2) # Anzeigen der Anzahl der Elemente in der Liste data


An der Zahl der Elemente in der Liste erkenen sie, dass es eine andere Datei sein muss, die gerade geladen wurde. 

Neben der Bibliothek ```json```gibt es auch eine spezielle Bibliothek für ndjson, die es erlaubt NDJSON so wie reguläres JSON zu laden. Installieren wir diese Bibliothek zur Sicherheit zuerst

In [ ]:
! pip3 install ndjson

In [ ]:
import ndjson

with open(fname, 'r') as f:
    data = ndjson.load(f)

len(data) # Anzeigen der Anzahl der Elemente in der Liste data  


Sollten Sie einen Fehler erhalten, liegt das wahrscheinlich daran, dass ihre Python Version erfordert explizit zu deklarieren wie die Datei codiert ist. Fügen Sie den Parameter ```encoding='utf-8``` der Funktion ```open()```hinzu, d.h.:

```Python
with open(fname, 'r', encoding='utf-8'):
    data = ndjson.load(f)
```

## Nach dem Laden der Daten: Inspizieren

JSON ist nicht so gut lesbar für Menschen. Einfacher ist es oft JSON als Tabelle zu betrachten. Das können wir mit Pandas machen. Tabellen heißen hier DataFrame. 
    

In [ ]:
df = pd.DataFrame(data) # Erstellen eines Pandas DataFrames aus der Liste data
df.head(3) # Anzeigen der ersten 3 Zeilen des DataFrames


Znächst lohnt es sich zu betrachten welche Spalten es in der Tabelle gibt. Dies können wir uns in der oben ausgedruckten Tabellenansicht ansehen oder mit dem Befehl ```df.columns``` anzeigen lassen.

In [ ]:
df.columns

In der Diskussion im Kurs haben wir herausgearbeitet, dass die Spalten der Tabelle irgendwie nicht den Erwartungen an Instagram Posts entsprechen. So fehlen beispielsweise die Medieninhalte, Textunterschriften, Autor*innen etc. Dafür gibt es Spalten wie ```timestamp_collected```die weniger auf Instagram als auf die Situation der Datenerfassung mit Zeeschuimer zu verweisen scheinen. 

Diese Beobachtung bestätigt sich bei der genaueren Betrachtung einer Beispielzelle in der Spalte ```data```. Diese beginnt mit ```{```. Damit ist der Inhalt ein Dictionary, das selbst als Tabele verstanden und betrachtet werden kann. Um derartig verschachtelte JSON Daten leichter in Tabellen zu konvertieren, bietet pandas die Funktion ```pd.json_normalize()```.

In [ ]:
df_normalized = pd.json_normalize(data) # Normalisieren des DataFrames
df_normalized.head(3) # Anzeigen der ersten 3 Zeilen des normalisierten DataFrames

In [ ]:
df_normalized.columns

Die normalisierte Tabelle hat 497 Spalten. Pandas zeigt in der Tabellenansicht nur eine definierte Zahl dieser Spalten. Um alle anzuzeigen, muss die Einstellung geändert werden.

In [ ]:
# Show all columns in pandas
pd.set_option('display.max_columns', None)

df_normalized.head(3) # Anzeigen der ersten 3 Zeilen des normalisierten DataFrames

Beim scrollen durch die Daten fällt auf, dass es weiterhin Zellen gibt, die komplexe Daten beinhalten. Um nur diese anzuzeigen, können wir folgendes schreiben:

In [ ]:
# Die Spalten, die in der Ausgabe angezeigt werden sollen, können explizit in eckigen Klammern angegeben werden
# Die doppelten eckigen Klammern sind notwendig, um eine Liste von Spaltennamen zu übergeben.

df_normalized[['data.id','data.carousel_media']].head(3) 

Jede Zelle der Spalte data.carousel_media enthält eine Liste von Dictionaries. Diese können wir erneut als eine Tabelle denken. Um die Werte einer Zelle in ```data.carousel_media``` als Tabelle auszugeben können wir folgendes tun:
m die Inhalte von ```data.carousel_media``` als Tabelle anzuzeigen können wir folgendes tun:

In [ ]:
cell_value = df_normalized['data.carousel_media'].to_list()[0]
media_table = pd.json_normalize(cell_value)

In [ ]:
len(media_table.columns)

Die Tabelle ```media_table``` enthält 9 Zeilen mit 41 Spalten. Jede der Zeilen bezieht sich auf einen Post so wie er in der ```df_normalized``` in einer Zeile abgebildet ist. Die Spalten ```data.id``` in df_normalized und ```carousel_parent_id``` in der Medientabelle scheinen übereinzustimmen. Das können wir wie fotgt überprüfen.

In [ ]:
data_id_value = df_normalized['data.id'].to_list()[0]
len(media_table[media_table['carousel_parent_id'] == data_id_value])


Der oben stehende Code filtert die Tabelle ```media_table``` und zeigt nur die Inhalte, bei denen die Zelle ```carousel_parent_id``` itentisch ist mit dem Wert der Variable ```data_id_value```. Da die Tabelle immer noch 9 Zeilen beinhaltet, können wir darauf schließen, dass unsere Intuition stimmt. Die Tabelle ```media_table```enthält also einen Verweis auf den Post, so dass wir die Tabellen auch auseinanderziehen können. Aktuell enthält die Tabelle aber nur Medieninformationen zu einem Post. Um die zu allen Posts gehörenden Informaitonen zu erhalten, müssen wir eine Liste von Listen von Dictionaries in einen Dataframe konvertieren. ```pd.json_normalize()```kann nur mit Listen von Dictionaries umgehen, so dass wir unsere Daten erst konvertieren müssen. 

GitHub Copilot schlägt auf der Grundlage des Kommentars ```# Konvertiere eine Liste von Listen von Dictionaries in eine Liste von Dictionaries``` folgendes Beispiel vor.

In [ ]:
# Konvertiere eine Liste von Listen von Dictionaries in eine Liste von Dictionaries

data = [[{'a': 1, 'b': 2}, {'a': 3, 'b': 4}], [{'a': 5, 'b': 6}, {'a': 7, 'b': 8}]]
data_flat = [item for sublist in data for item in sublist] # Dies ist die entscheidende Codezeile, die wir übernehmen können.
data_flat

Passen wir den Code nun für unser Beispiel an. Dazu speichern wir die Werte der Spalte ```data.carousel_media```zuerst in eine Variable ```column_data```als  Liste und wenden danach den Befehl ```[item for sublist in data for item in sublist]``` an. Dazu müssen dir nur ```data``` durch ```column_data``` ersetzen.

In [ ]:
column_data = df_normalized['data.carousel_media'].to_list()
column_data_flat = [item for sublist in column_data for item in sublist]
media_table = pd.json_normalize(column_data_flat)

Ganz funktioniert unsere Lösung noch nicht. Ein Element in ```column_data``` scheint keine Liste zu sein. Mit der Hilfe von Copilot fügen ich eine ```if```Bedingung hinzu, die sicherstellt, dass jedes übernommene Element eine Liste ist. 

Hierzu eine Hintergrundinformation: Leere Tabellenzellen bleiben in Pandas nicht einfach Leer, sondern beinhalten einen sogenannten Nullwert ```NaN```. Diese Nullwerte verursachen den Fehler. 

In [ ]:
column_data_flat = [item for sublist in column_data if isinstance(sublist, list) for item in sublist]
media_table = pd.json_normalize(column_data_flat)
media_table.head(3)

In [ ]:
print(f'Die Medientabelle hat {len(media_table)} Zeilen und {len(media_table.columns)} Spalten.')

Bis zu diesem Punkt haben wir einen JSON Datensatz geöffnet und diesen auf seine Struktur hin analysiert. Die erstellte Tabelle können wir mit ```pd.to_csv()``` speichern. CSV-Dateien sind ein Austauschformat für tabellarische Daten. Die Extension bedeutet "Comma-Separated-Values", d.h. dass unrrschiedliche Tabellenzellen durch Kommata getrennt werden. In der Praxis enthalten CSV Dateien oft auch andere Trennzeichen, z.B. Semikolon oder Tabulatoren. Ich verwende selbst gern Tabulatoren, da dies die reinen Textdateien etwas besser lesbar für mich macht. Hierzu übergebe ich den Paramater ```sep='\t'``` beim speichern. Grundsätzlich können Sie CSV Dateien aber mit Tabellenverarbeitungsprogrammen importieren. Excel und Google Srpeadseets hat hierfür eigene Importfunktionen. Wenn sie CSV Dateien einfach mit diesen Programmen öffnen, klappt die Tabellenansicht meistens nicht.

In [ ]:
df_normalized.to_csv('../00_data/zeeschuimer-#gegengewalt-2024-05-07.csv', index=False, sep='\t') # Speichern des DataFrames als CSV-Datei
media_table.to_csv('../00_data/zeeschuimer-#gegengewalt-2024-05-07-media.csv', index=False, sep='\t') # Speichern der Medientabelle als CSV-Datei

## Zeehaven

Für die Konvertierung von Zeeshuimer JSON-Daten in eine CSV Datei gibt es ein Online Tool. Unter https://publicdatalab.github.io/zeehaven/können sie datauf zugreifen. Wir haben gesehen, dass Zeeschuimer einen komlexen Datensatz zurückgibt. Schauen wir an wie Zeehaven mit diesen Daten umgeht. 

In [ ]:
zeehaven_file = '../00_shared_data/Zeeshuimer/zeehaven–zeeschuimer-#gegengewalt-2024-2024-05-13T06_09_11.003Z.csv'
zeehaven_df = pd.read_csv(zeehaven_file)

In [ ]:
zeehaven_df.head(3)

In [ ]:
len(zeehaven_df.columns)

Die Tabelle, die wir mit ```pd.json_normalize()``` aus den Zeeschuimer NDJSON Daten erzeugt haben, enthält 497 Spalten. Zeehaven gibt eine Tabelle zurück, die 912 Spalten enthält. Hier wurden eitens der Entwickler des Konvertierungstools Entscheidungen getroffen, die sich möglicherweise auf die spätere Analyse der Daten auswirken. Dies ist keinesfalls problematisch. Im Gegenteil! Jedes Forschen beruht auf Entscheidungen, die mögliche Erkenntnispfade eröffnen und andere verschhließen. Wichtig ist also nicht dass Entscheidungen getroffen werden, sondern vielmehr, dass wir uns dieser bewusst werden und üebr diese reflektieren.